## **TML Assignment 4 - Task 3** ##

## **Ahrar Bin Aslam and Muhammad Mubeen Siddiqui**

## Installing Lime

In [1]:
!pip install lime --q

## Importing Libraries and Setup

In [ ]:
import os
import requests
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from lime.wrappers.scikit_image import SegmentationAlgorithm
from sklearn.linear_model import Ridge
import pickle

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("imagenet_images", exist_ok=True)
os.makedirs("lime_outputs", exist_ok=True)

## Downloading Images

In [ ]:
# Image URLs
image_urls = {
    "West_Highland_white_terrier": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02098286_West_Highland_white_terrier.JPEG",
    "American_coot": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02018207_American_coot.JPEG",
    "racer": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n04037443_racer.JPEG",
    "flamingo": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02007558_flamingo.JPEG",
    "kite": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01608432_kite.JPEG",
    "goldfish": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01443537_goldfish.JPEG",
    "tiger_shark": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01491361_tiger_shark.JPEG",
    "vulture": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01616318_vulture.JPEG",
    "common_iguana": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n01677366_common_iguana.JPEG",
    "orange": "https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n07747607_orange.JPEG",
}

# Download images
for name, url in image_urls.items():
    path = f"imagenet_images/{name}.jpeg"
    if not os.path.exists(path):
        r = requests.get(url)
        with open(path, 'wb') as f:
            f.write(r.content)

## Loading the Model and Image Transforms

In [ ]:
# Load model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval().to(device)

# Transforms
def transform_for_lime(img):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    return transform(img).permute(1, 2, 0).numpy()

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## Classifier for LIME

In [8]:
# Batch classifier
def batch_predict(images):
    processed = []
    for img in images:
        if img.dtype != np.uint8:
            img = (img * 255).astype(np.uint8)

        if img.shape[0] == 1 and img.shape[1] == 1 and img.shape[2] == 3:
            img = np.tile(img, (224, 224, 1))  # if somehow super small

        img_pil = Image.fromarray(img)
        tensor = preprocess(img_pil)
        processed.append(tensor)

    batch = torch.stack(processed, dim=0).to(device)
    with torch.no_grad():
        logits = model(batch)
    return torch.nn.functional.softmax(logits, dim=1).cpu().numpy()

# Init explainer
explainer = lime_image.LimeImageExplainer()
lime_params_dict = {}

# Loop through images
for name in image_urls:
    print(f"Explaining: {name}")
    img_path = f"imagenet_images/{name}.jpeg"
    pil_img = Image.open(img_path).convert('RGB')
    img_np = transform_for_lime(pil_img)

    # Get top predicted class
    input_tensor = preprocess(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model(input_tensor)
    top_label = int(pred.argmax().item())

    hide_color = np.quantile(img_np, 0.2, axis=(0, 1)).astype(int).tolist()

    # Segmentation
    segmentation_fn = SegmentationAlgorithm(
        'quickshift',
        kernel_size=5,
        max_dist=150,
        ratio=0.3
    )

    # Run Lime Explainer
    explanation = explainer.explain_instance(
        img_np,
        classifier_fn=batch_predict,
        labels=(top_label,),
        top_labels=1,
        num_features=12,             
        num_samples=700,              
        batch_size=10,
        segmentation_fn=segmentation_fn,
        distance_metric='cosine',
        model_regressor=Ridge(alpha=0.5),
        random_seed=42,
        hide_color=hide_color
    )

    # Visualize explanation
    temp, mask = explanation.get_image_and_mask(
        label=top_label,
        positive_only=True,
        num_features=10,
        hide_rest=False
    )

    temp_uint8 = (temp * 255).astype(np.uint8) if temp.dtype != np.uint8 else temp
    output_img = mark_boundaries(temp_uint8, mask)

    plt.figure(figsize=(6, 6))
    plt.imshow(output_img)
    plt.axis('off')
    plt.title(f"LIME - {name}")
    plt.tight_layout()
    plt.savefig(f"lime_outputs/{name}_lime.png")
    plt.close()

    # Save params for Deliverable 4
    lime_params_dict[name] = {
        'labels': (top_label,),
        'hide_color': hide_color,
        'top_labels': 1,
        'num_features': 12,
        'num_samples': 700,
        'batch_size': 10,
        'segmentation_fn': segmentation_fn,
        'distance_metric': 'cosine',
        'model_regressor': Ridge(alpha=0.5),
        'random_seed': 42
    }

# Save the dictionary as pickle for submission
with open("lime_params.pkl", "wb") as f:
    pickle.dump(lime_params_dict, f)

print("LIME visualizations and lime_params.pkl created.")

Explaining: West_Highland_white_terrier


  0%|          | 0/700 [00:00<?, ?it/s]

Explaining: American_coot


  0%|          | 0/700 [00:00<?, ?it/s]

Explaining: racer


...

Explaining: common_iguana


  0%|          | 0/700 [00:00<?, ?it/s]

Explaining: orange


  0%|          | 0/700 [00:00<?, ?it/s]

LIME visualizations and lime_params.pkl created successfully.


In [9]:
import requests

# Path to your .pkl file (generated from your script)
file_path = "lime_params.pkl"

# Your provided token
token = "93145372"

# Submit to server
response = requests.post(
    "http://34.122.51.94:9091/lime",
    files={"file": open(file_path, "rb")},
    headers={"token": token}
)

# Print server response
print(response.json())

{'avg_iou': 0.3206986647682452, 'avg_time': 3.362541437149048}
